In [1]:
import optuna
from optuna.pruners import SuccessiveHalvingPruner

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from lion_pytorch import Lion

print("cuda", torch.cuda.is_available())  
print(torch.cuda.get_device_name(0)) 

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pytorch_lightning.trainer.connectors.data_connector")
warnings.filterwarnings("ignore", category=UserWarning, module="lightning_fabric.plugins.environments.slurm")

torch.cuda.empty_cache()

from utils.train import MoleculeModel, MoleculeDataModule, GATv2Model, get_metric, save_trial_to_csv, create_hyperopt_dir
from utils.prepare import FeaturizationParameters, MoleculeDataset, MoleculeData

import logging

logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)


cuda True
NVIDIA GeForce RTX 3080


In [2]:
molecule_dataset = torch.load("../data/QM_100.pt")

In [3]:
num_workers = 8
in_features = molecule_dataset[0].x.shape[1]
edge_attr_dim = molecule_dataset[0].edge_attr.shape[1]
max_epochs = 100
patience = 5

### Гиперпараметры

In [4]:
import optuna

def adaptive_callback(study, trial):
    completed_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])

    if len(completed_trials) > 0:
        layer_sizes = []
        for t in completed_trials:
            layer_sizes.extend([t.params[k] for k in t.params.keys() if "size" in k])

        min_layer_size = min(layer_sizes)
        max_layer_size = max(layer_sizes)

        study.set_user_attr("min_layer_size", max(32, min_layer_size - 32)) 
        study.set_user_attr("max_layer_size", min(512, max_layer_size + 32)) 

def objective(trial):
    min_layer_size = trial.study.user_attrs.get("min_layer_size", 64)
    max_layer_size = trial.study.user_attrs.get("max_layer_size", 256)

    try:


        num_preprocess_layers = trial.suggest_int('num_preprocess_layers', 9, 9)
        preprocess_hidden_features = [trial.suggest_int(f'preprocess_layer_{i}_size', min_layer_size, max_layer_size) for i in range(num_preprocess_layers)]
        
        num_postprocess_layers = trial.suggest_int('num_postprocess_layers', 2, 2)
        postprocess_hidden_features = [trial.suggest_int(f'postprocess_layer_{i}_size', min_layer_size, max_layer_size) for i in range(num_postprocess_layers)]

        num_heads = [trial.suggest_int(f'num_heads_{i}', 16, 32, step=2) for i in range(2)]

        dropout_rates = [trial.suggest_float(f'dropout_rate_{i}', 0.0, 0.1) for i in range(num_preprocess_layers + 2 + num_postprocess_layers)]
        use_batch_norm = [trial.suggest_categorical(f'use_batch_norm_{i}', [True]) for i in range(num_preprocess_layers + 2 + num_postprocess_layers)]
        learning_rate = 2.2e-5
        weight_decay = 3e-5
        step_size = 80
        gamma = 0.2
        batch_size = 64

        #learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-3, log=True)
        #weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        #step_size = trial.suggest_int('step_size', 10, 200, step=10)
        #gamma = trial.suggest_float('gamma', 0.1, 0.9)
        #batch_size = trial.suggest_int('batch_size', 64, 128, step=64)

        # Создание модели с переменными гиперпараметрами
        base_model = GATv2Model(
            atom_in_features=in_features,
            edge_in_features=edge_attr_dim,
            num_preprocess_layers=num_preprocess_layers,
            preprocess_hidden_features=preprocess_hidden_features,
            num_heads=num_heads,
            dropout_rates=dropout_rates,
            activation_fns=[nn.ReLU for _ in range(len(dropout_rates))],  # ReLU для всех слоев
            use_batch_norm=use_batch_norm,
            num_postprocess_layers=num_postprocess_layers,
            postprocess_hidden_features=postprocess_hidden_features,
            out_features=1
        )

        model = MoleculeModel(
            base_model=base_model,
            optimizer_class=Lion,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            step_size=step_size,
            gamma=gamma,
            batch_size=batch_size,
            metric='rmse'
        )

        # Обучение модели
        data_module = MoleculeDataModule(molecule_dataset, batch_size=128, num_workers=num_workers)
        early_stop_callback = EarlyStopping(monitor="val_loss", patience=patience, mode="min")

        trainer = pl.Trainer(
            max_epochs=max_epochs,
            devices=1,
            accelerator='gpu',
            logger=False,
            enable_progress_bar=False,
            enable_checkpointing=False,
            enable_model_summary=False,
            
            callbacks=[early_stop_callback]
        )
        trainer.fit(model, data_module)

        val_loss = trainer.callback_metrics["val_loss"].item()

        save_trial_to_csv(trial, hyperopt_dir, val_loss)

    except RuntimeError as e:
        if 'CUDA out of memory' in str(e):
            print("CUDA out of memory. Skipping this trial.")
            return float('inf')
        raise  

    return val_loss

torch.set_float32_matmul_precision('medium')

hyperopt_dir = create_hyperopt_dir()
print(f"Results will be saved in: {hyperopt_dir}")

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner())

study.optimize(objective, n_trials=10000, callbacks=[adaptive_callback])

print(f'Best trial: {study.best_trial.number}')
print(f'Best value (RMSE): {study.best_trial.value}')
for key, value in study.best_trial.params.items():
    print(f'{key}: {value}')


[I 2024-03-24 13:04:46,368] A new study created in memory with name: no-name-1911dddc-b2ff-4a83-98f3-bbbe0033fd8e


Results will be saved in: hyperopt_2


[I 2024-03-24 13:04:58,143] Trial 0 finished with value: 0.20607195794582367 and parameters: {'num_preprocess_layers': 9, 'preprocess_layer_0_size': 183, 'preprocess_layer_1_size': 205, 'preprocess_layer_2_size': 207, 'preprocess_layer_3_size': 199, 'preprocess_layer_4_size': 96, 'preprocess_layer_5_size': 80, 'preprocess_layer_6_size': 158, 'preprocess_layer_7_size': 126, 'preprocess_layer_8_size': 214, 'num_postprocess_layers': 2, 'postprocess_layer_0_size': 149, 'postprocess_layer_1_size': 223, 'num_heads_0': 24, 'num_heads_1': 24, 'dropout_rate_0': 0.031962656640796984, 'dropout_rate_1': 0.02956168019603861, 'dropout_rate_2': 0.014682575191738656, 'dropout_rate_3': 0.06398784623336592, 'dropout_rate_4': 0.008140409311788577, 'dropout_rate_5': 0.08135092047468634, 'dropout_rate_6': 0.047936130500189404, 'dropout_rate_7': 0.09579297722132185, 'dropout_rate_8': 0.035816038881370804, 'dropout_rate_9': 0.09592622655042783, 'dropout_rate_10': 0.03769124335553854, 'dropout_rate_11': 0.031

FileNotFoundError: [Errno 2] No such file or directory: 'hyperopt_2/optuna_results.csv'